Preparing Simplified Street Graph
==============================
Start with this file.

It downloads the raw data from OSM and prepares a simplified street graph, consisting of one node for every intersection and one edge for every street segment. This simplified street graph is also referred to as *centerline graph*. The centerline graph also gets enriched with additional data sources such as public transit routes, elevation or traffic volumes. The resulting data is saved in the *process* folder and can be used by other scripts for further work.

In [ ]:
import warnings
warnings.filterwarnings('ignore')


import os
import shutil

import snman
from snman.constants import *
from snman import osmnx_customized as oxc

PERIMETER = 'db'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
export_path = os.path.join(data_directory, 'outputs', PERIMETER)

#CRS_internal = 29119    # for Boston
#CRS_internal = 32216    # for Chicago
CRS_internal = 2056      # for Zurich
CRS_for_export = 4326
oxc.settings.useful_tags_way = OSM_TAGS

Loading data
------------
Loads the necessary datasets, including downloading the raw OSM data

In [ ]:
print('Load perimeters')
perimeters = snman.io.load_perimeters(
    os.path.join(inputs_path, 'perimeters', 'perimeters.shp'),
    crs=CRS_internal
)

print('Get data from OSM server')
G_raw, G = snman.io.create_street_graph_from_OSM(
    perimeters.to_crs(4326).loc[PERIMETER]['geometry'],
    CRS_internal,
    return_raw=True
)

In [ ]:
if 1:
    print('Export raw street graph')
    # each street is one edge, the lanes are saved as an attribute
    snman.io.export_street_graph(
        G_raw,
        os.path.join(process_path, 'raw_street_graph_edges.gpkg'),
        os.path.join(process_path, 'raw_street_graph_nodes.gpkg'),
        crs=CRS_for_export
    )

print('Load manual intersections')
# polygons used to override the automatically detected intersections in some situations
given_intersections_gdf = snman.io.load_intersections(
    os.path.join(inputs_path, 'intersection_polygons/intersection_polygons.gpkg'),
    crs=CRS_internal
)

if 1:
    print('Load public transit routes')
    pt_routes = snman.io.load_public_transit_routes_zvv(
        "C:/Users/lballo/polybox/BUSINESS/PROJECTS/01 Dornbirn GVK/DATA/Buslinien Dornbirn.gpkg",
        perimeter=perimeters.loc[PERIMETER]['geometry']
    )

if 1:
    print('Load parking spots')
    parking_spots = snman.io.load_parking_spots(
        "C:/Users/lballo/polybox/BUSINESS/PROJECTS/01 Dornbirn GVK/DATA/Parkflächen Erhebung Stadt+OSM_punkte.gpkg",
        crs=CRS_internal
    )

if 0:
    print('Load street polygons')
    street_polygons = snman.io.load_street_polygons_zurich(
        os.path.join(inputs_path, 'switzerland', 'zurich', 'av', 'av_ktzh_bodenbedeckung_polygons_2.gpkg'),
        crs=CRS_internal
    )

if 1:
    print('Load widths from an official network')
    widths = snman.io.import_geofile_to_gdf(
        'C:/Users/lballo/polybox/BUSINESS/PROJECTS/01 Dornbirn GVK/DATA/strassennetz_mit_breiten_bereinigt.gpkg',
        crs=CRS_internal
    )
    widths['width'] = widths['fb_1_avgw']

Enriching the street graph before simplification
------------------------------------------------
Enrichment steps that need to be done before simplification

In [ ]:
if 1:
    print('Identify hierarchy')
    # split the edges into hierarchy categories, such as main roads, local roads, etc.
    snman.hierarchy.add_hierarchy(G)
    
if 1:
    print('Add parking spaces')
    snman.enrichment.match_parking_spots(G, parking_spots)

Simplification
--------------
Consolidates intersections and merges edges so that we obtain a centerline graph. The process needs to be repeated a few times to catch all secondary simplification possibilities

In [ ]:
G, intersections_gdf = snman.simplification.simplify_street_graph(
    G, given_intersections_gdf,
    verbose=True, iterations=3,
    edge_geometries_simplification_radius=10,
    exclude_edge_hierarchies_from_simplification={snman.hierarchy.HIGHWAY}
)

Updating pre-calculated attributes
----------------------------------
Updates the OSM tags and stats like aggregate lane widths to match the simplified graph

In [ ]:
if 1:
    snman.space_allocation.normalize_cycling_lanes(G)
    
if 1:
    snman.street_graph.organize_edge_directions(G)

In [ ]:
print('Add lane stats to edges')
# how many lanes, how wide, etc.
snman.space_allocation.generate_lane_stats(G)

print('Update OSM tags')
# to match the simplified and merged edges
snman.space_allocation.update_osm_tags(G)

if 0:
    print('Update street counts per node')
    spn = oxc.stats.count_streets_per_node(G, nodes=G.nodes)
    nx.set_node_attributes(G, values=spn, name="street_count")

Enrichment
----------
Add additional data to the centerline graph, such as public transit, elevation, and traffic counts

In [ ]:
if 0:
    print('Add street widths by fitting the edges to street polygons')
    snman.fitting.fit_edges(G, street_polygons)
else:
    for uvk, data in G.edges.items():
        data['width'] = data[KEY_LANES_DESCRIPTION + '_width_total_m']

In [ ]:
if 1:
    print('Match widths from the provided official network')
    snman.enrichment.match_width(
        G, widths, 
        max_dist=200, max_dist_init=500, max_lattice_width=5
    )

In [ ]:
if 1:
    print('Add public transit')
    snman.enrichment.match_public_transit_zvv(
        G, pt_routes,
        max_dist=200, max_dist_init=400, max_lattice_width=20,
        route_number_column='line_name', direction_column='direction'
    )

In [ ]:
if 1:
    print('Add elevation')
    G = oxc.elevation.add_node_elevations_raster(
        G,
        #os.path.join(inputs_path, 'switzerland', 'switzerland', 'ch_dhm_25', 'ch_dhm_2056.tif'),
        #raster_crs=2056,
        os.path.join(inputs_path, 'copernicus_dem', 'Copernicus_DSM_10_N47_00_E009_00', 'DEM', 'Copernicus_DSM_10_N47_00_E009_00_DEM.tif'),
        raster_crs=4326, #Chicago
        graph_crs=CRS_internal,
        cpus=1
    )
    G = oxc.elevation.add_edge_grades(G, add_absolute=False)

if 0:
    traffic_counts_npvm = snman.io.load_traffic_counts_npvm(
        os.path.join(inputs_path, 'traffic_volumes/npvm_2017_filtered.gpkg')
    )
    snman.enrichment.match_traffic_counts_npvm(G, traffic_counts_npvm)
if 1:
    snman.street_graph.add_edge_costs(G)

Export
------
Save the datasets to the hard drive. All files can be opened in QGIS using the *snman_detailed.qgz* file.

In [ ]:
if 1:
    print('Export street graph')
    # each street is one edge, the lanes are saved as an attribute
    snman.io.export_street_graph(
        G,
        os.path.join(process_path, 'street_graph_edges.gpkg'),
        os.path.join(process_path, 'street_graph_nodes.gpkg'),
        crs=CRS_for_export
    )

if 1:
    print('Export lane geometries')
    # each lane has an own geometry and with as an attribute, for visualization purposes
    L = snman.lane_graph.create_lane_graph(G)
    snman.io.export_lane_geometries(
        L,
        os.path.join(process_path, 'lane_geometries_edges.gpkg'),
        os.path.join(process_path, 'lane_geometries_nodes.gpkg'),
        scaling=1, crs=CRS_for_export
    )

if 1:
    print('Save intersection geometries into a file')
    snman.io.export_gdf(
        intersections_gdf,
        os.path.join(process_path, 'intersections_polygons.gpkg'),
        columns=['geometry'], crs=CRS_for_export
    )